In [499]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

In [504]:
class Layer:
    def __init__(self,features,neurons,activationfunc):
        self.features            = features
        self.neurons             = neurons
        self.activationfunc      = activationfunc
        self.weight              = np.random.randn(self.neurons, self.features)
        self.bias                = np.random.randn(neurons,1)
        self.input               = np.zeros([self.features,1])
        self.output              = np.zeros([self.features,1])
        self.activation          = np.zeros([self.features,1])

    def activationReLu(self,inputs):
        return np.maximum(inputs,0)
        
    def derivativeReLu(self,inputs):
        return inputs > 0
    
    def activationSoftMax(self, inputs):
        exp_values = np.exp(inputs - np.max(inputs))
        return exp_values / np.sum(exp_values)
        
    def derivativeSofMax(self):
        return self.activation*(1-self.activation)
    
    def derivativeFunction(self):
        if self.activationfunc   == "relu":
            return self.derivativeReLu(self.output)
        elif self.activationfunc == "softmax":
            return self.derivativeSofMax()
        else :
            return 1
    
    def activationFunction(self,inputs):
        if self.activationfunc   == "relu":
            return self.activationReLu(inputs)
        elif self.activationfunc == "softmax":
            return self.activationSoftMax(inputs)
        else:
            return inputs

    def feedforward(self,input):
        self.input = input
        self.output      = np.dot(self.weight, input)+self.bias
        self.activation  = self.activationFunction(self.output)  
        return self.activation
        
    def backpropagation(self,dZ):
        dEdZ = dZ*self.derivativeFunction()
        dW   = np.dot(dEdZ, self.input.transpose())
        dB   = dEdZ
        dE   = np.dot(self.weight.transpose(), dEdZ)
 
        return dW, dB, dE

    def learn(self, dW, dB, learning_rate):
        self.weight -= dW*learning_rate
        self.bias   -= dB*learning_rate

class MLP:
    def __init__(self, input_dim:int, layer_dims:list[int], activationfuncs:list[int], learning_rate=0.001):
        self.learning_rate = learning_rate
        self.network = []
        self.network.append(Layer(input_dim, layer_dims[0], activationfuncs[0]))
        for i in range(len(layer_dims)-1):
            self.network.append(Layer(layer_dims[i], layer_dims[i+1], activationfuncs[i]))

    def feedforward(self, input):
        out = input
        for i in range(len(self.network)):
            out = self.network[i].feedforward(out)        
        return out

    def backpropagation(self, ground_truth, output):
        error = output - ground_truth

        for i in reversed(range(len(self.network))):
            dW, dB, error = self.network[i].backpropagation(error)
            self.network[i].learn(dW, dB, self.learning_rate)

    def mse_loss(self,ground_truth, output):
        return ((ground_truth - output)**2)/2

In [505]:
# Load the dataset
data = load_iris()

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.3, random_state=42)

# Convert the training and testing sets into input and labels
train_input = X_train
test_input = X_test

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler on the training data and transform both the training and testing data
train_input = scaler.fit_transform(train_input)
test_input = scaler.transform(test_input)


# One-hot encode the target labels
encoder = OneHotEncoder()
train_labels = encoder.fit_transform(y_train.reshape(-1, 1)).toarray()
test_labels = encoder.transform(y_test.reshape(-1, 1)).toarray()


In [506]:
mlp = MLP(4, layer_dims = [64, 64, 3], activationfuncs=[ "softmax", "softmax"], learning_rate=0.007)

In [507]:
epoch=500

for i in range(epoch):
    outs = []
    for x, y in zip(train_input, train_labels):
        out = mlp.feedforward(x.reshape(4,1))
        mlp.backpropagation(y.reshape(3,1), out)
        outs.append(out)

In [508]:
outs= []
for input, label in zip(train_input, y_train):
    out = mlp.feedforward(input.reshape(4,1))
    outs.append(np.argmax(out))

In [509]:
np.array(outs).transpose()

array([1, 2, 2, 2, 2, 1, 2, 1, 0, 2, 1, 0, 0, 0, 1, 2, 0, 0, 0, 1, 0, 1,
       2, 0, 1, 1, 0, 2, 1, 1, 1, 2, 1, 0, 1, 2, 0, 0, 1, 2, 0, 2, 0, 0,
       2, 1, 2, 1, 2, 2, 1, 0, 0, 1, 2, 0, 0, 0, 1, 2, 0, 2, 2, 0, 2, 2,
       2, 2, 2, 0, 2, 1, 2, 1, 1, 2, 0, 2, 1, 0, 1, 2, 2, 0, 1, 2, 2, 0,
       2, 0, 2, 2, 2, 1, 2, 1, 2, 1, 2, 0, 1, 1, 0, 1, 2], dtype=int64)

In [510]:
y_train-outs

array([ 0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  1,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0, -1,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,
        0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1, -1,  0, -1,
        0,  0,  0,  0,  0,  0,  0, -1,  0, -1,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0, -1,  1,  0,  0,  0,  1,
        0,  0,  0], dtype=int64)

In [511]:
outs= []
for input, label in zip(test_input, y_test):
    out = mlp.feedforward(input.reshape(4,1))
    # print(out)
    outs.append(np.argmax(out))


y_test-outs

array([ 0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,
        0,  1, -1,  0,  0,  0,  1, -1, -1,  0,  0], dtype=int64)

In [512]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

# Load the wine dataset
data = load_wine()
X = data.data   # Features (inputs)
y = data.target # Target (outputs)

# Split the dataset into a training set and a test set
# test_size=0.2 means 20% of data is used for testing and 80% for training
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Now, X_train and y_train are the training inputs and outputs,
# and X_test and y_test are the testing inputs and outputs.
train_input = X_train
test_input = X_test

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler on the training data and transform both the training and testing data
train_input = scaler.fit_transform(train_input)
test_input = scaler.transform(test_input)

# One-hot encode the target labels
encoder = OneHotEncoder()
train_labels = encoder.fit_transform(y_train.reshape(-1, 1)).toarray()
test_labels = encoder.transform(y_test.reshape(-1, 1)).toarray()

print(train_labels.shape)

epoch=100

mlp = MLP(13, layer_dims = [128, 64, 3], activationfuncs=["softmax", "softmax"], learning_rate=0.05)

for i in range(epoch):
    outs = []
    for x, y in zip(train_input, train_labels):
        out = mlp.feedforward(x.reshape(13,1))
        mlp.backpropagation(y.reshape(3,1), out)
        outs.append(out)
    # print(mlp.mse_loss(y, out))

outs= []
for input, label in zip(train_input, y_train):
    out = mlp.feedforward(input.reshape(13,1))
    # print(out)
    outs.append(np.argmax(out))



(142, 3)


In [513]:
np.array(outs).transpose()

array([2, 2, 1, 2, 0, 1, 1, 2, 2, 0, 1, 1, 2, 0, 1, 0, 0, 2, 2, 1, 1, 0,
       1, 0, 2, 1, 1, 2, 0, 0, 0, 2, 0, 0, 1, 2, 1, 0, 2, 2, 0, 2, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 2, 1, 1, 1, 0, 1, 1, 1, 2, 2, 0, 1, 2, 2, 1,
       1, 0, 1, 2, 2, 1, 2, 1, 1, 1, 0, 0, 2, 0, 2, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 2, 1, 1, 2, 2, 2, 1, 0, 0, 1, 2, 2, 0, 1, 2, 2, 2, 2, 1,
       0, 1, 0, 2, 0, 0, 1, 0, 0, 2, 1, 0, 2, 2, 0, 0, 2, 2, 2, 1, 1, 1,
       1, 1, 1, 2, 0, 1, 1, 0, 1, 1], dtype=int64)

In [514]:
y_train-outs

array([ 0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0], dtype=int64)

In [515]:
outs= []
for input, label in zip(test_input, y_test):
    out = mlp.feedforward(input.reshape(13,1))
    # print(out)
    outs.append(np.argmax(out))


y_test-outs

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)